In [1]:
# comment out to run on drive + colab

# ! pip install transformers_interpret

In [2]:
# from google.colab import drive
#
# drive.mount('/content/drive/')

In [3]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers_interpret import MultiLabelClassificationExplainer
import torch
from transformers import pipeline
from datasets import load_from_disk

import warnings
warnings.filterwarnings('ignore')

In [4]:
# comment out the filepath to run on drive + colab
filepath = "../models/scenario_level/control_non_aphasic"
# filepath = "/content/drive/Shareddrives/AphasiaProject/models/scenario_level/conduction_control"

model = AutoModelForSequenceClassification.from_pretrained(filepath+"/model")
tokenizer = AutoTokenizer.from_pretrained(filepath+"/tokenizer", padding=True,
                                          truncation=True, return_tensors="pt",
                                          max_length=512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if device == torch.device("cuda"):
    model.to(device)
pipe = pipeline("text-classification", model=filepath+"/model", tokenizer=filepath+"/tokenizer", device=device, truncation=True, padding=True)
cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)

cuda


In [5]:
model.config

DistilBertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "../models/scenario_level/control_non_aphasic/model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "CONTROL",
    "1": "NOT APHASIC"
  },
  "initializer_range": 0.02,
  "label2id": {
    "CONTROL": 0,
    "NOT APHASIC": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.1",
  "vocab_size": 30528
}

In [6]:
model.config.id2label

{0: 'CONTROL', 1: 'NOT APHASIC'}

In [7]:
def interpret(data):
    for text, label in zip(list(data["new_preprocessed_text"]), list(data["label"])):
        if device == torch.device("cuda"):
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512).input_ids.to(device)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")
        else:
            inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", add_special_tokens=True, max_length=512)
            # tranformers interpret gives error for input of 512 tokens
            if len(inputs[0]) != 512:
                with torch.no_grad():
                    logits = model(**inputs).logits

                predicted_class_id = logits.argmax().item()
                print("True: ", label, "Pred: ", model.config.id2label[predicted_class_id])
                print("Text: ", text)
                cls_explainer(text)
                cls_explainer.visualize()
                print("----------------------------------------------------------------------")

In [8]:
scenarios = ["Speech", "Important_Event", "Cinderella", "Stroke", "Cat"]
# make sure not to interpret on trained scenarios
texts = load_from_disk(filepath + "/dataset")["test"].remove_columns(["Unnamed: 0", "input_ids", "attention_mask"]).to_pandas()
texts = texts.dropna()
texts["label"] = [model.config.id2label[x] for x in texts["label"]]
n = 3

In [9]:
texts

,label,new_preprocessed_text,scenario
0,CONTROL,okay well UP1 little sally was out riding her ...,Cat
1,CONTROL,oh alright. okay. this is a classic. little jo...,Umbrella
2,CONTROL,okay. so there was a man that had a daughter. ...,Cinderella
3,CONTROL,FP1 well first there was a little boy. and he ...,Window
4,CONTROL,take two slices of bread out out. FP1 get out ...,Sandwich
...,...,...,...
862,NOT APHASIC,ah. well again little johnnys FP1 probably goi...,Umbrella
863,CONTROL,FP1 cinderellas father FP1 remarries FP1 a wom...,Cinderella
864,CONTROL,well the cats up in the tree. they get stuck i...,Cat
865,CONTROL,mhm FP1 i can think about the day that i got e...,Important_Event


# Speech scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [10]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[0])].sample(n=n)])

In [11]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  what are choices. well. probably not good. when you say when you say the ability to tell a story like from like creative. if the will the pictures be making a picture and i will just tell you what it is. probably. no.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  i think i do fairly well. i think very well. not fluently.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.63),CONTROL,3.01,[CLS] i think i do fairly well . i think very well . not fluent ##ly . [SEP]
n/a,(0.35),NOT APHASIC,-3.22,[CLS] i think i do fairly well . i think very well . not fluent ##ly . [SEP]


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  how do i what. very well. excellent. well i have noone to talk yiddish to so i do not know whether i should include that. well i i can speak some yiddish. but there is nobody to converse with so i do not know whether that would qualify.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  it depends on what i am talking about. FP1 if it is pretty typical about you know where i was what i am doing as opposed to what i am thinking or talking about something from the past. telling a story. i will put it that way. if i told you something about what happened it becomes i i it is like i cannot find the words but i cannot come to the explanation either. and FP1 people will say. FP1 even my mom will be like. i do not know what you are saying. so you know what i mean. my words are in place so to speak but they do not make sense. yes yes. mm mhm. and it a lot of times i cannot come up with the words either. but what i have learned quickly to do is how to define what the word is which then becomes part of telling a story in order to point out what the word is.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  FP1 i think it is been better. FP1 UP1 my memory is is not so good. and i think it is partly of my partly FP1 my aphasia. and the other part is FP1 old age.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  FP1 my speech is pretty good today you know.


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
n/a,(0.47),CONTROL,1.29,[CLS] FP1 my speech is pretty good today you know . [SEP]
n/a,(0.53),NOT APHASIC,-1.62,[CLS] FP1 my speech is pretty good today you know . [SEP]


----------------------------------------------------------------------


# Important event scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [12]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[1])].sample(n=n)])

In [13]:
interpret(data)

True:  NOT APHASIC Pred:  NOT APHASIC
Text:  okay. well that is that is a good FP1 question. bebecause FP1 i met firstnamed FP1 if if i did not have the stroke i never would have met firstnamed and same for her. we we made we have FP1 met each other through FP1 through FP1 health connections. FP1 and FP1 that was in back in two thousand and six we met. and we have been together since then. and as a matter of fact i was going to FP1 i was involved in a clinical trial out of university of pennsylvania. and FP1 they they put FP1 a stimulator in my outer tissue on my brain. so they put it in. i went down there for six weeks doing FP1 rkl therapy FP2 heavy very FP1 hard physical therapy. then after six weeks they took it out. and long and short of it is that before i went down i i said to firstnamed. bebecause we were FP1 talking on the phone. and and so i says. how about if we meet before i start the the FP1 the FP1 the FP1 the the study. so we we met. and FP1 that is that is FP1 history. 

----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  mhm. FP1 the main thing that happened in our family is our younger daughter which was twenty eight yesterday. when she was five years old she had FP1 chicken pox after her sister. and right when she was going to be recovering and going to be able to go to school again it was kind of like the last days of school so she did not go to school. anyways FP1 i noticed one day FP1 she was UP1 not walking. and i was FP1 carrying her. and i am like. what am i carrying this five year old girl. and she had what they call ataxia FP1. because the blood got an infection or something. FP1 but she did recover from it. FP1 perfect. mhm.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  CONTROL
Text:  which story. well when i got married the first time i hadta FP1 plan my whole wedding myself because mom was much older. and she left it up to me. and FP1 i went home and then went back to lawrence. i got married in lawrence FP1 in danforth chapel. and FP1 my husband FP2 my fianc at that time came down and my folks came up. and we had dinner and FP1 then we went to back to the hotel. and FP1 my uncle married me. yeah he is a methodist methodist minister. and his wife took over the bathroom and did not get let my sister or i in there. and FP1 so then we went to the chapel and FP1 on the way FP1 my brother inlaw was driving and he almost wrecked us. so i think that will be enough of that story.


----------------------------------------------------------------------


# Cinderella scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [14]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[2])].sample(n=n)])

In [15]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  cinderella is one of four sisters. and she is very hard working at home. she scrubs floors. basically the other three sisters seem like they have been given the right to not do anything. well one of the sisters gets a invitation to the ball. so they are all invited to go. and cinderella cannot go. for some reason they do not tell her. or they did not let her go. anyways she is heartbroken. so she wants to go to the ball. but she has nothing to wear. so in some way she ends up crying. and she is her fairy godmother comes out of nowhere. and FP1 has her fitted with a gown a beautiful gown. and FP1 she has no way of getting to the ball. and gets her new FP1 slippers for the ball. FP1 but the the mean sisters thought she was going to stay home. and unexpectedly she is able to go to the ball. FP1 transportation actually is a pumpkin that is turned into a carriage. and so she is beautifully dressed. so she goes to the ball. and FP1 this prince that was th

----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  a long time ago in a land far away there was a little girl and her dad. i believe her mom had passed away. and her dad ended up marrying another woman with two daughters. and they brought the families together. and FP1 the stepmother favored her children over cinderella and gave her children all the privileges and made cinderella work day in and day out. and FP1 the prince was having a ball to choose his future wife. and they received an invitation at all the households in the land. and when the time came the wicked stepmother would not give cinderella would not let cinderella go. she had to do the chores. but her daughters could go. and they had beautiful dresses. but she banished cinderella into doing chores. and cinderella was dreaming that she could go to the ball. and she had a fairy godmother appear. and FP1 she turned her garb into a nice ballgown and the horse into FP2 a 

----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  FP1 well cinderella FP1 is working for FP1 them. and them is i think FP1 the mother and the two little girls at the beginning. and then they grow up and she is still working cinderella. but that but only the one that is growling you know. but otherwise and then FP1 they look she he. no here i go again. cinderella looks out and sees the FP1 the castle. and FP1 and she in her mirror she sees a FP1 a picture a picture. and she i do not know how she gets out but she wears it. and they says they say. she is going to the the ball with us yeah. anyhow she the prince invites her to dance. FP1 it would be fine. this dress you can take out and wear. and but you hafta be back by midnight. and FP1 and so FP1 when FP1 the prince she looks at the clock and realizes it is just you know a couple minutes before twelve. and she hurries up to run. and she drops FP1 her her shoe fell off. an

----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  FP1 cinderellas father FP1 married a woman who had two children. and so FP1 the FP1 her cinderellas two stepsisters and her stepmother made cinderella clean up FP1 after FP1 everything. she hadta wash the floor. and FP1 one day the prince FP2 send around flyer flyers FP1 to cinderellas cinderellas house FP1 UP1 oh FP1 UP1 to offer them to come to his FP1 his ball. and FP1 UP1 so the the two stepsisters FP1 were not going to allow cinderella to come. FP1 so after they left for the ball FP1 cinderellas FP1 fairy godmother came and FP1 uFP2 UP1 wished her into a a pretty ball gown. and the pumpkin pumpkin changed into a carriage. and the rats or mice turned into FP1 the horses. and her fairy godmother told her she hadta be back home by noon i mean by midnight. and so FP1 she went in the the carriage. and FP1 at the ball she met the prince. and they dance. and then the clock 

----------------------------------------------------------------------
True:  NOT APHASIC Pred:  CONTROL
Text:  alright. well cinderella was young. she was given to her FP1 mother and two evil stepsisters. they are they are all evil i suppose. she grew up there. she is working like a dog. she befriends not the cat but she befriends other animals. FP1 the king he is beside himself wants FP1 his son to get married. i do not know why. i i do not. FP1 and so the boy sends around a boy man sends around a proclamation that FP1 he is going to marry somebody he meets at the party. but before that cinderella goes back in there. and she throws together a gown. and it looks dynamite. and she comes out. and the evil stepdaughters FP1 buried her tear that tear pearls off her necks. not pearls. they are green emeralds off her neck. so she goes outside and is weeping weeping. and she is visited by her fairy godmother. and FP1 fairy godmother waves her wand. and the mice and so forth are transferred i

----------------------------------------------------------------------


# Stroke scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [16]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    df = texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])]
    # no controls in stroke scenario
    if len(df) >= n:
        data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[3])].sample(n=n)])

In [17]:
interpret(data)

True:  NOT APHASIC Pred:  NOT APHASIC
Text:  FP1 october thirtieth or thirty first of last year FP1 two thousand eleven. not very far after my birthday. FP1 well i remember going to houston. my FP1 my doctors are there. and FP1 i went for the lvad checkup. and FP1 i had i had a little trouble saying things like FP1 lots of main strokes. and then FP1 they kept me in the hospital. and then the next morning i woke up and i did not have any speech. hmhm. well FP1 UP1 well my mom freaked out. and FP1 i did not i did not see there was a problem with it until. yeah until. yeah. mhm. FP1 i have been to speech therapy. FP1 well that is about it. yeah yeah mhm.


----------------------------------------------------------------------


# Cat scenario interpretation
Three examples for conduction and three for control (# of examples defined by n)

In [18]:
data = pd.DataFrame(columns=['source_file','scenario','preprocessed_text','label'])

for x in model.config.label2id.keys():
    data = pd.concat([data, texts[(texts["label"] == x) & (texts["scenario"] == scenarios[4])].sample(n=n)])

In [19]:
interpret(data)

True:  CONTROL Pred:  CONTROL
Text:  okay. so there was a dog. and he was chasing a cat. and the cat belonged to a little girl. so the cat was trying to get away from the dog. and it ran into a tree. and the girl was saying. help help. i need someone to get my cat out of the tree. so a man that was walking by got his ladder. and he tried to climb up and get the cat. but the dog knocked the ladder down. so then he could not get down either. so she called the fire department. and they came and helped get the man down and the cat.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  UP1 okay FP1 sally and her dad were FP1 walking around the park one day. sally was riding her tricycle. and FP1 as they were coming along they noticed that a dog chased a cat up a tree. and her dad FP1 she urged her dad to help the cat. and FP1 so FP1 he fetched a ladder and climbed up the tree. and then the ladder fell down because the dog knocked it over. and FP1 some neighbors nearby saw that this was going on so they called the FP1 fire department. and sure enough the fire department came to FP1 help retrieve the dad from the tree and the cat. and at the end of the day every one was fine.


----------------------------------------------------------------------
True:  CONTROL Pred:  CONTROL
Text:  this story is about my daughter. she is reaching up trying to find or get her cat which climbed the tree. in order to help her i took my ladder and climbed a tree. but for some reason the ladder fell down. and so i am stuck up in the tree. and our puppy dog decided that was pretty funny. and he was barking at us. my wife looked out the window. and she saw the predicament i was in. so she called the local fire department. and they came out with their ladders. and were able to get me down from the tree and also my daughters cat. the puppy dog enjoyed the whole thing. end of story.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  the FP1 a a family was outside. and the cat went up the tree. and the dad went up there trying to rescue him. he he brought a ladder. and the letter fell down. and somehow the firemen got called. and and they they FP1 they safe the man. and all was good again. i i i botched that story.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  NOT APHASIC
Text:  well FP1 the man went up to FP2 it shook took a ladder and went up to FP2 save the cat. and FP1 he was able to FP1 and the ladder fell. and he was not able to get down. so he hadta FP1 get the firemen to FP1 get him down. and FP1 and that is that was FP1 was a dog barking. and the the girl is trying to FP1 save the cat. and and he is he is FP1 he is stuck.


----------------------------------------------------------------------
True:  NOT APHASIC Pred:  CONTROL
Text:  well it looks like a lot of things have happened. and there is i am going to hafta guess at what were all these things happened. but there is a man in the tree. there is a cat in the tree. there is a bird in the tree. there is a dog barking at the tree. and there is a young child reaching up into the tree. they are fireman running towards the tree. so i am guessing that there was a bird in the tree. the cat may have run up the tree after the bird. FP1 perhaps the young girl maybe wanted the cat out of the tree. so then i am guessing a man tried to help by climbing up the tree on a ladder. and then the ladder fell over once he was in the tree. so now he is stuck in the tree. i think the dog is upset. and just perhaps maybe maybe the guy came over with the dog. and now the dogs upset his owners in the tree. and somebody either called or the fireman saw that the man was stuck. a

----------------------------------------------------------------------
